In [87]:
from math import *
import point_similarity as ps
from scipy.integrate import quad
from sympy import *

In [44]:
#based on an x0, y0 and x1, y1, split the path between them into sections, then for each point, use it to complete qudratic to a certain point
'''
x0, y0 = 4,7.5
x1, y1 = 5,8
p, q = 7,8
t = -24.5-4*b
b = ((q + 24.5 - 2*p*p)/(p-4))
b
'''

'\nx0, y0 = 4,7.5\nx1, y1 = 5,8\np, q = 7,8\nt = -24.5-4*b\nb = ((q + 24.5 - 2*p*p)/(p-4))\nb\n'

In [10]:
def calc_coeff(x, y, p, q):
    c1 = y -2*x*x #-2 can be replaced by whatever G is
    b = ((q - c1 - 2*p*p)/(p-x))
    c = c1 - x * b
    return b, c

In [46]:
calc_coeff(3, 7, 7, 8)

(-19.75, 48.25)

In [31]:
x = symbols('x')

In [43]:
b,c = calc_coeff(3, 7, 7, 8)
f = 2*x*x + b*x + c
f_prime = diff(f, x)
print(f_prime)

4*x - 19.75


In [61]:
def derivative(b, c):
    x = symbols('x')
    f = 2*x*x + b*x + c
    f_prime = diff(f, x)
    return f_prime

In [67]:
derivative(-19.75, 48.25)

4*x - 19.75

In [96]:
der = derivative(-19.75, 48.25)
function = sqrt(1 + der * der)
integrate(function, (x, 3, 7))

16.8341378512273

In [109]:
def curve_length(x0, y0, p, q):
    b, c = calc_coeff(x0, y0, p, q)
    der = derivative(b, c)
    f = sqrt(1 + der * der)
    dist = integrate(f, (x, x0, p))
    return dist

In [110]:
curve_length(3, 7, 7, 8)

16.8341378512273

In [118]:
def distance_from_base(x0, y0, x1, y1):
    pair1 = [x0, y0]
    pair2 = [x1, y1]
    distance = dist(pair1, pair2)
    return distance + 10 # 10 is the set distance from the base to the lateral root

In [117]:
distance_from_base(3, 7, 5, 8)

12.23606797749979

In [124]:
def total_cost(alpha, x0, y0, x1, y1, p, q):
    cost = alpha * curve_length(x0, y0, p, q) + (1 - alpha) * (curve_length(x0, y0, p, q) + distance_from_base(x0, y0, x1, y1))
    return cost

In [125]:
total_cost(0.5, 3, 7, 5, 8, 7, 8)

22.9521718399772

In [ ]:
## def best_cost() todo: iterate through many points on the lateral root to minimize total cost